In [1]:
import rasterio
import geopandas as gpd
import glob
import zipfile

In [2]:
gdf_scms = gpd.read_file('data/folhas-20-oumais-quadriculas-treinamento.gpkg').to_crs(5533)

In [3]:
gdf_quadriculas = gpd.read_file('data/quadriculas-32m-nas-areas-de-favela-atual.gpkg', layer='quadriculas-de-treinamento').to_crs(5533)

In [4]:
gdf_edificacoes = gpd.read_file('data/edificacoes-nas-quadriculas.gpkg')

In [5]:
gdf_scm_quadriculas = gpd.sjoin(gdf_scms, gdf_quadriculas, how='right')

In [6]:
gpd.sjoin(gdf_quadriculas, gdf_edificacoes, how='right')

,index_left,left,top,right,bottom,id,CLASSID,FEATID,DATA_CRIAC,ESCALA,ALTURA_EDI,COD_SETOR_,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4875.0,2007-03-11,1:1000,5.632,034,"MULTIPOLYGON (((334195.394 7392885.796, 334197..."
1,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4876.0,2007-03-11,1:1000,5.532,034,"MULTIPOLYGON (((334198.192 7392873.669, 334193..."
2,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4878.0,2007-03-11,1:1000,7.072,034,"MULTIPOLYGON (((334181.198 7392874.087, 334177..."
3,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4879.0,2007-03-11,1:1000,7.392,034,"MULTIPOLYGON (((334177.987 7392867.815, 334183..."
4,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4675.0,2007-03-11,1:1000,7.190,034,"MULTIPOLYGON (((334165.920 7392841.071, 334162..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90031,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2810892.0,2007-03-11,1:1.000,2.890,062,"MULTIPOLYGON (((340932.652 7397339.384, 340941..."
90032,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815655.0,2007-03-11,1:1000,4.620,033,"MULTIPOLYGON (((334015.626 7392801.673, 333996..."
90033,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815661.0,2007-03-11,1:1000,7.750,033,"MULTIPOLYGON (((334088.563 7392770.539, 334052..."
90034,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815662.0,2007-03-11,1:1000,3.119,033,"MULTIPOLYGON (((334029.463 7392827.406, 334029..."


In [163]:
from rasterio.windows import Window
from rasterio.transform import Affine
import rasterio.features
import math

In [188]:
folder = 'ortofotosMDC2004/treinamento'

# Interando sobre cada folha SCM
for scm in gdf_scms.qo2004_cod.to_list():
    
    print(scm)

    # DEscomprime o ZIP correspondente a folha SCM
    with zipfile.ZipFile(f'{folder}/1O_{scm}.zip', 'r') as zip_ref:
        zip_ref.extractall(folder)

    # Abre o raster SCM
    dataset = rasterio.open(f'{folder}/1O_{scm}_060803.JPG')

    # Cria dataset de mascara das edificacoes
    gdf_edif = gpd.clip(gdf_edificacoes, gdf_scms.loc[gdf_scms.qo2004_cod == scm].geometry)
    edif = rasterio.features.rasterize(shapes=gdf_edif.unary_union, out_shape=dataset.shape, default_value=1, transform=dataset.transform)

    profile = dataset.profile
    profile.update({
        'count':1,
        'driver':'GTiff',
        "compress":"LZW",
        'crs':'EPSG:5533'
    })
    profile.pop('photometric')

    with rasterio.open(f'ortofotosMDC2004/edificacoes-scm/{scm}-edificacoes-mask.tif', 'w', **profile) as dst:
        # Read the data from the window and write it to the output raster
        dst.write(edif, 1)

    edif = rasterio.open(f'ortofotosMDC2004/edificacoes-scm/{scm}-edificacoes-mask.tif')

    # Intera sobre cada quadricula da folha SCM Atual
    for row in gdf_scm_quadriculas.loc[gdf_scm_quadriculas.qo2004_cod == scm].itertuples():
        
        # Janela de 256x256
        xoff = (row.geometry.bounds[0] - dataset.bounds.left)/.125
        yoff = (row.geometry.bounds[3] - dataset.bounds.top)/.125
        
        window = Window(xoff, abs(yoff), 256, 256)
        # transform = dataset.window_transform(window)
        transform = Affine.translation(row.geometry.bounds[0], row.geometry.bounds[3]) * Affine.scale(.125, -.125)
        # transform = Affine(.125, 0., (xoff * 0.125) + dataset.bounds.left, 0., -.125, dataset.bounds.bottom + (yoff * 0.125))

        profile = dataset.profile
        profile.update({
            'height': 256,
            'width': 256,
            'crs':'EPSG:5533',
            'transform': transform})

        with rasterio.open(f'ortofotosMDC2004/licoes/{scm}-{row.Index}.JPG', 'w', **profile) as dst:
            # Read the data from the window and write it to the output raster
            dst.write(dataset.read(window=window))

        # Edificacoes na determinada janela
        # print(profile)
        profile.update({
            'count':1,
            'driver':'GTiff',
            "compress":"LZW"
        })
        profile.pop('photometric')
        profile.pop('interleave')
        with rasterio.open(f'ortofotosMDC2004/licoes/{scm}-{row.Index}-mask.tif', 'w', **profile) as dst:
            dst.write(edif.read(window=window))

    
    # break

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 256, 'height': 256, 'count': 1, 'crs': 'EPSG:5533', 'transform': Affine(0.125, 0.0, 317068.69199464074,
       0.0, -0.125, 7382463.274485611), 'tiled': False, 'compress': 'LZW'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 256, 'height': 256, 'count': 1, 'crs': 'EPSG:5533', 'transform': Affine(0.125, 0.0, 317068.6919872083,
       0.0, -0.125, 7382433.274522935), 'tiled': False, 'compress': 'LZW'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 256, 'height': 256, 'count': 1, 'crs': 'EPSG:5533', 'transform': Affine(0.125, 0.0, 317068.69195004896,
       0.0, -0.125, 7382283.274709538), 'tiled': False, 'compress': 'LZW'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 256, 'height': 256, 'count': 1, 'crs': 'EPSG:5533', 'transform': Affine(0.125, 0.0, 317068.69194261683,
       0.0, -0.125, 7382253.274746854), 'tiled': False, 'compress': 'LZW'}
{'driver': 'GTiff', 'dtype': 'uin